In [1]:
import pybedtools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [29]:
data = pybedtools.BedTool("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Promotor_with_TFBS/All_GTEx_Prom_with_TFBS.bed")

# Visu von einzelnen TFBS um Überblick zu geben
- Häufigkeit TFBS unique im gesamten Datensatz
- Häufigkeit der Vorkommen jedes TFBS pro Promotor

In [42]:
from get_TFBS_pair import get_tfbs_subset, get_information_for_singel_tfbs_set

tfbs = get_tfbs_subset(data, ["SMAD3"]).saveas()

In [27]:
def get_orientation_for_single_tfbs_in_pct(tfbs_name, tfbs_arr, tfbs_orientation):
    """
    Calculates the percentage of nT Orientation for a specific tfbs.
    """
    # get List of orientation of tfbs
    tfbs_orientation_per_prom = []
    for idx, orientation in enumerate(tfbs_orientation): 
        if tfbs_arr[idx] == tfbs_name:
            tfbs_orientation_per_prom.append(orientation)
    # Calculate percentage of nT in Orientation List. 
    pct_nT = np.sum(np.array(tfbs_orientation_per_prom)=="nT")/len(np.array(tfbs_orientation_per_prom))

    return pct_nT

In [52]:
def get_count_of_single_tfbs(data, tfbs):

    tfbs_BedTool = get_tfbs_subset(data, [tfbs]).saveas()
    count_prom = tfbs_BedTool.count()

    count_tfbs = np.array([])
    nT_percent = np.array([])
    for BedTool_Interval in tfbs_BedTool:
        chr, geneID, tfbs_arr, tfbs_close_tss_arr, tfbs_dist_tss_arr, tfbs_orientation, tfbs_count, tfbs_unique_count = get_information_for_singel_tfbs_set(BedTool_Interval)
        # get array for tfbs count per region
        count_tfbs_per_prom = np.sum(tfbs_arr == tfbs)
        count_tfbs = np.append(count_tfbs, count_tfbs_per_prom)
        # get percentage of nT Orienation per region
        nT_pct = get_orientation_for_single_tfbs_in_pct(tfbs, tfbs_arr, tfbs_orientation)
        nT_percent = np.append(nT_percent, nT_pct)

    return count_prom, count_tfbs, nT_percent

In [53]:
count_prom, count_tfbs, nT_percent = get_count_of_single_tfbs(data, "SMAD3")